In [1]:
from collections import Counter
from datetime import timedelta, datetime
import glob
from itertools import chain
import json
import os
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
#import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import tensorflow as tf

In [2]:
directory = 'D:/ANACONDA/envs/tf-gpu/code/NLP/kakao/data/'
font_path = directory + 'NanumGothic.ttf'
font_name = fm.FontProperties(fname=font_path, size=10).get_name()
plt.rc('font', family=font_name, size=12)
plt.rcParams["figure.figsize"] = (20, 10)
register_matplotlib_converters()

In [6]:
# MetaData preprocessing

In [3]:
metadata = pd.read_json(directory + 'metadata.json', lines=True)
metadata.shape # (643104, 9)
metadata.head()

,article_id,display_url,id,keyword_list,magazine_id,reg_ts,sub_title,title,user_id
0,782,https://brunch.co.kr/@bookdb/782,@bookdb_782,"[여행, 호주, 국립공원]",8982,1474944427000,세상 어디에도 없는 호주 Top 10,"사진으로 옮기기에도 아까운, 리치필드 국립공원",@bookdb
1,81,https://brunch.co.kr/@kohwang56/81,@kohwang56_81,"[목련꽃, 아지랑이, 동행]",12081,1463092749000,,[시] 서러운 봄,@kohwang56
2,4,https://brunch.co.kr/@hannahajink/4,@hannahajink_4,[],0,1447997287000,무엇 때문에,무엇을 위해,@hannahajink
3,88,https://brunch.co.kr/@bryceandjuli/88,@bryceandjuli_88,"[감정, 마음, 위로]",16315,1491055161000,,싫다,@bryceandjuli
4,34,https://brunch.co.kr/@mijeongpark/34,@mijeongpark_34,"[유럽여행, 더블린, 아일랜드]",29363,1523292942000,#7. 내 친구의 집은 어디인가,Dubliner#7,@mijeongpark


In [4]:
magazine = pd.read_json(directory + 'magazine.json', lines=True)

In [12]:
magazine[:3]

,id,magazine_tag_list
0,38842,"[브런치북, 육아일기, 대화법, 들려주고픈이야기]"
1,11540,"[tea, food]"
2,11541,[food]


In [19]:
magazine_keyword = []
for x in metadata['magazine_id']:
    if(x != 0 ):
        if(list(magazine[magazine['id']==x]['magazine_tag_list']) != []):
            magazine_keyword.append(list(magazine[magazine['id']==x]['magazine_tag_list'])[0])
        else:
            magazine_keyword.append([])
    else:
        magazine_keyword.append([])

In [20]:
len(magazine_keyword)

643104

# Keyword_list Embedding

In [36]:
keywordlist = metadata['keyword_list'].copy()
text_keyword = []
# metadata keyword
for x_list in keywordlist:
    if len(x_list) != 0:
        for x in x_list:
            text_keyword.append(x)
# magazine_keyword
for x_list in magazine_keyword:
    if len(x_list) != 0 :
        for x in x_list:
            text_keyword.append(x)
            
print('text_keyword : ',len(text_keyword))


text_keyword :  3077864


In [37]:
# make dictionary
# generate dataset, dictionary for word
word_to_id = {}
id_to_word = {}

for word in text_keyword:
    if word not in word_to_id:
        new_id = len(word_to_id)
        word_to_id[word] = new_id
        id_to_word[new_id] = word
# 공백 표시할 어휘 : eos
word_to_id['eos'] = len(word_to_id)
id_to_word[len(word_to_id)-1] = 'eos'
# vocab에 없는 단어는 UNK로 표시
word_to_id['UNK'] = len(word_to_id)
id_to_word[len(word_to_id)-1] = 'UNK'

In [44]:
corpus_kr = []
for i in range(len(keywordlist)):
    if len(keywordlist[i]) != 0:
        for x in keywordlist[i]:
            corpus_kr.append(x)
    if len(magazine_keyword[i]) != 0 :
        for x in magazine_keyword[i]:
            corpus_kr.append(x)
    if len(keywordlist[i]) != 0:
        for x in keywordlist[i]:
            corpus_kr.append(x)
    if len(magazine_keyword[i]) != 0 :
        for x in magazine_keyword[i]:
            corpus_kr.append(x)
    if len(keywordlist[i]) != 0:
        for x in keywordlist[i]:
            corpus_kr.append(x)
    if len(magazine_keyword[i]) != 0 :
        for x in magazine_keyword[i]:
            corpus_kr.append(x)
    if len(keywordlist[i])!=0 or len(magazine_keyword[i]) != 0:
        for x in range(4):
            corpus_kr.append('eos')
            
len(corpus_kr)

11725560

In [45]:
corpus = [word_to_id[w] for w in corpus_kr]

In [51]:
corpus[:15]

[0, 1, 2, 410, 769, 6627, 0, 1, 2, 410, 769, 6627, 0, 1, 2]

In [52]:
# word_to_id, id_to_word, corpus

vocabulary_size = len(word_to_id) # ~~= 96864

# title 은 생략

In [65]:
# title 
josa1 = ['이','가','을','를','에','은','는','께','로','의','과','와','고','아','야','도','다']
josa2 = ['께서','에서','한테','더러','으로','보다','처럼','만큼','같이','조차','마저','까지','부터','이다','에도']
josa3 = ['으로는','께서는','한테는','처럼은','만큼은','같이는','마저도','조차도','까지도','습니다',]

titlelist = metadata['title'].copy()

text_title = []
for x_title in titlelist:
    if len(x_title) != 0:
        for space in list(['.',',','/','@','-','·','?','...','\xa0']):
            x_title = x_title.replace(space,' ')
        for remov in ['#','^','*','[',']','(',')',"'",'"']:
            x_title = x_title.replace(remov,'')
        x_title.lower()
        words = x_title.split(' ')
        for word in words:
            if len(word) != 0:
                if word[-3:] in josa3:
                    word = word[:-3]
                if len(word) != 0 :
                    if word[-2:] in josa2:
                        word = word[:-2]
                    if len(word) != 0:
                        if word[-1] in josa1:
                            word = word[:-1]
                text_title.append(word)
print('text_title : ',len(text_title))
word_to_id_title = word_to_id.copy()
id_to_word_title = id_to_word.copy()

text_title :  2490429


# Tensorflow skip, word2vec 구현

In [54]:
# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
import collections
def generate_batch(data, batch_size, num_skips, skip_window, data_index):
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window

    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)

    span = 2 * skip_window + 1                      # context = skip_window + target + skip_window
    assert span > num_skips

    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)   # 다음 단어 인덱스로 이동. len(data) = 17005207

    for i in range(batch_size // num_skips):

        targets = list(range(span))     # 1. 0부터 span-1까지의 정수로 채운 다음
        targets.pop(skip_window)        # 2. skip_window번째 삭제
        np.random.shuffle(targets)      # 3. 난수를 사용해서 섞는다.

        start = i * num_skips
        batch[start:start+num_skips] = buffer[skip_window]

        for j in range(num_skips):
            labels[start+j, 0] = buffer[targets[j]]

        # 새로운 요소가 들어가면서 가장 먼저 들어간 데이터 삭제
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)

    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels, data_index

In [55]:
import math

In [56]:
# Step 4: skip-gram 모델 구축
np.random.seed(1)
tf.set_random_seed(1)

batch_size = 128        # 일반적으로 16 <= batch_size <= 512
embedding_size = 128    # embedding vector 크기
skip_window = 1         # target 양쪽의 단어 갯수
num_skips = 2           # 컨텍스트로부터 생성할 레이블 갯수

valid_size = 16     # 유사성을 평가할 단어 집합 크기
valid_window = 100  # 앞쪽에 있는 분포들만 뽑기 위한 샘플
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # negative 샘플링 갯수

train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

truncated = tf.truncated_normal([vocabulary_size, embedding_size],
                                stddev=1.0 / math.sqrt(embedding_size))
nce_weights = tf.Variable(truncated)
nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

# embeddings 벡터.
embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
embed = tf.nn.embedding_lookup(embeddings, train_inputs)

# 배치 데이터에 대해 NCE loss 평균 계산
nce_loss = tf.nn.nce_loss(weights=nce_weights,
                          biases=nce_biases,
                          labels=train_labels,
                          inputs=embed,
                          num_sampled=num_sampled,
                          num_classes=vocabulary_size)
loss = tf.reduce_mean(nce_loss)

# SGD optimizer
optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

# 유사도를 계산하기 위한 모델. 학습 모델은 optimizer까지 구축한 걸로 종료.
norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
normalized_embeddings = embeddings / norm
valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)

W0717 16:03:12.183416 17092 deprecation.py:323] From D:\ANACONDA\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0717 16:03:12.395745 17092 deprecation.py:506] From <ipython-input-56-be20dda57574>:41: calling reduce_sum_v1 (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [65]:
embeddings.shape

TensorShape([Dimension(96864), Dimension(128)])

In [64]:
similarity.shape

TensorShape([Dimension(16), Dimension(96864)])

In [57]:
# Step 5: skip-gram 모델 학습
# 4시 4분 시작
num_steps = 1000001
data = corpus
ordered_words = list(word_to_id.keys())
with tf.Session() as session:
    session.run(tf.global_variables_initializer())

    average_loss, data_index = 0, 0
    for step in range(num_steps):
        batch_inputs, batch_labels, data_index = generate_batch(data, batch_size, num_skips, skip_window, data_index)

        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}
        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val

        # 마지막 10000번에 대한 평균 loss 표시
        if step % 10000 == 0:
            if step > 0:
                average_loss /= 10000
            print('Average loss at step {} : {}'.format(step, average_loss))
            average_loss = 0
        
        # 10만번째마다 valid size만큼 sim 계산
        if step % 100000 == 0:
            sim = similarity.eval()         # (16, vocab_size)

            for i in range(valid_size):
                valid_word = ordered_words[valid_examples[i]]

                top_k = 8
                nearest = sim[i].argsort()[-top_k - 1:-1][::-1]
                log_str = ', '.join([ordered_words[k] for k in nearest])
                print('Nearest to {}: {}'.format(valid_word, log_str))

    final_embeddings = normalized_embeddings.eval()
    saver = tf.train.Saver()
    saver.save(session,'keyword_embedding_model')


Average loss at step 0 : 324.4151611328125
Nearest to 출판: 아빠육아, 청소년노동, 인간관계도, 전략실행력, 이호연, 옥잠화, 제품디자이너, 산수유축제
Nearest to 아이돌: 오늘질문, 가든, 파라과이, 전몽각, 터키리라, 미네랄온천, 만족하는연습, 야행축제
Nearest to 생각: 방해금지모드, 감정다툼, 텃새, 원칙에의한경영, 바이오연료, 교육프로그램, 제주도출판사, 친노
Nearest to 사회복지사: 정치도서, 제니퍼, 교육강좌, 서예대전, 이지훈기자, 하도급계약, 무크지, 오사카여행
Nearest to 방송연예: 비폭력운동, 숫돌, 인간로봇, 퇴장, 첩보, 수사관, 롯데하이마트, 국력
Nearest to 부동산경매변호사: 브라질치안, 에바항공, 녹차콜라, 제중원, 투스카니의태양, 원어민교사, 항일, 코르크마개
Nearest to 단상: 법정통역, 책마을, 부자운세, 중국중국, 노동착취, 디센던트, 다코타패닝, 감상
Nearest to 교육: 내맘대로영화읽기, 딸기바구니, 매화축제, 국민드론, 우리동네, 라면국물티백, 바보상자, 기온
Nearest to 취업: 아메리카여행, 로잔선언, 최소한, 스포티지r, 오염, 충동인류, 싱글오일, 사천시
Nearest to 마케팅: 윙크, 인생정리, 돈나무, 세운상가, 철학서적, 주사바늘, 앰네스티, 실사출력물
Nearest to 방송: 한민족, 복합체, 명란계란말이, 운동모임, 아이나비, 딱정벌레, 마음으로소통하다, 문무대왕릉
Nearest to 감성에세이: 신재생에너지, 협상바이블, 지산락페, 정기과정, 철원기행, 상상력가동, 놀이법, 엄마감정
Nearest to 하이에나: 힌트, 설매재, 전쟁기자, 언차티드, 도시서점, 미크로네시아, 지방소멸, 엔드
Nearest to 버스: 나고야메시, 인권조례, 정월대보름행사, 전두환, 마을공동체미디어, 스키장오픈, 제주스노쿨링명소, 뎅기열
Nearest to 행복: 운두령, 관계부사, 고당봉, 스티브, 화물연대, 모아이석상

In [58]:
print(final_embeddings.shape)
print(type(final_embeddings))
np.save(directory+'keyword_embedding_matrix',final_embeddings)

(96864, 128)
<class 'numpy.ndarray'>


In [66]:
atc = metadata.copy()
atc['reg_datetime'] = atc['reg_ts'].apply(lambda x : datetime.fromtimestamp(x/1000.0))
atc.loc[atc['reg_datetime'] == atc['reg_datetime'].min(), 'reg_datetime'] = datetime(2090, 12, 31)
atc['reg_dt'] = atc['reg_datetime'].dt.date
atc['type'] = atc['magazine_id'].apply(lambda x : '개인' if x == 0.0 else '매거진')
# 컬럼명 변경
atc.columns = ['id', 'display_url', 'article_id', 'keyword_list', 'magazine_id', 'reg_ts', 'sub_title', 'title', 'author_id', 'reg_datetime', 'reg_dt', 'type']
atc.head()
atc_cnt_by_reg_dt = atc.groupby('reg_dt', as_index=False)['article_id'].count()
read_file_lst = glob.glob(directory + 'read/read/*')
exclude_file_lst = ['read.tar']
read_df_lst = []
for f in read_file_lst:
    file_name = os.path.basename(f)
    if file_name in exclude_file_lst:
        print(file_name)
    else:
        df_temp = pd.read_csv(f, header=None, names=['raw'])
        df_temp['dt'] = file_name[:8]
        df_temp['hr'] = file_name[8:10]
        df_temp['user_id'] = df_temp['raw'].str.split(' ').str[0]
        df_temp['article_id'] = df_temp['raw'].str.split(' ').str[1:].str.join(' ').str.strip()
        read_df_lst.append(df_temp)
        
read = pd.concat(read_df_lst)
def chainer(s):
    return list(chain.from_iterable(s.str.split(' ')))
read_cnt_by_user = read['article_id'].str.split(' ').map(len)
read_raw = pd.DataFrame({'dt': np.repeat(read['dt'], read_cnt_by_user),
                         'hr': np.repeat(read['hr'], read_cnt_by_user),
                         'user_id': np.repeat(read['user_id'], read_cnt_by_user),
                         'article_id': chainer(read['article_id'])})
read_raw.shape
atc_read_cnt = read_raw[read_raw.article_id != ''].groupby('article_id')['user_id'].count()
atc_read_cnt = atc_read_cnt.reset_index()
atc_read_cnt.columns = ['article_id', 'read_cnt']
#metadata 결합
atc_read_cnt = pd.merge(atc_read_cnt, atc, how='left', left_on='article_id', right_on='article_id')
# metadata를 찾을 수 없는 소비 로그 제외
atc_read_cnt_nn = atc_read_cnt[atc_read_cnt['id'].notnull()]
# 소비수 기준 분류값
def get_class(x):
    if x >= 142:
        result = '5%'
    elif x >= 72:
        result = '10%'
    elif x >= 25:
        result = '25%'
    elif x >= 8:
        result = '50%'
    elif x >= 3:
        result = '75%'
    else:
        result = '100%'
    return result

atc_read_cnt_nn['class'] = atc_read_cnt_nn['read_cnt'].map(get_class)
#atc_read_cnt_nn.sort_values(by='read_cnt', ascending=False).head(2)

D:\ANACONDA\lib\site-packages\ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [73]:
total_top_50000 = atc_read_cnt_nn.sort_values(by='read_cnt', ascending=False)[:50000]
a = 20190222
a = pd.to_datetime(a,format='%Y%m%d')
c = 20901231
c = pd.to_datetime(c,format='%Y%m%d')
article_20190222_20190301 = atc_read_cnt_nn[atc_read_cnt_nn['reg_datetime'] >= a]
article_20190222_20190301 = article_20190222_20190301[article_20190222_20190301['reg_datetime'] != c]
recent_top_5000 = article_20190222_20190301.sort_values(by='read_cnt',ascending=False)[:5000]

In [76]:
b = 20190301
b = pd.to_datetime(b,format='%Y%m%d')
article_20190301_20190314 = atc[atc['reg_datetime'] >= b]
article_20190301_20190314 = article_20190301_20190314[article_20190301_20190314['reg_datetime'] != c]
len(article_20190301_20190314)

23821

In [78]:
total_top_50000.to_csv(directory+'total_top_50000.csv',encoding='utf-8')
recent_top_5000.to_csv(directory+'recent_top_5000.csv',encoding='utf-8')
article_20190222_20190301.to_csv(directory+'article_20190222_20190301.csv',encoding='utf-8')
article_20190301_20190314.to_csv(directory+'article_20190301_20190314.csv',encoding='utf-8')

In [ ]:
""" 읽을때는 아래와 같이
user_df = pd.read_csv(directory+'user_df.csv')
user_df = user_df.drop('Unnamed: 0',1)
"""

In [81]:
user_df = pd.read_csv(directory+'user_df.csv')
user_df = user_df.drop('Unnamed: 0',1)

In [136]:
user_df['read_cnt'].sum()

1526557

In [133]:
user_read_articles = set()
for x in user_df['read_articles']:
    x_li = ast.literal_eval(x)
    for y in x_li:
        user_read_articles.add(y)

In [143]:
a = final_embeddings[1]
a.shape

(128,)

In [144]:
a.dtype

dtype('float32')

In [148]:
user_vector = np.zeros(shape=(128,),dtype='float32')
type(user_vector)
user_vector.dtype

dtype('float32')

In [152]:
user_id = user_df['id'].tolist()
user_visit_cnt = user_df['visit_cnt'].tolist()
user_read_cnt = user_df['read_cnt'].tolist()
user_read_articles = user_df['read_articles'].tolist()

In [156]:
for i in range(len(user_read_articles)):
    x_li = ast.literal_eval(user_read_articles[i])
    user_read_articles[i] = x_li

In [165]:
from tqdm import tqdm

In [166]:
user_matrix = np.zeros(shape=(3000,128),dtype = 'float32')

In [169]:
len(user_matrix[1])

128

In [172]:
final_embeddings[word_to_id[word]]

TypeError: unhashable type: 'list'

In [183]:
read_list = user_read_articles[0]
article_id = read_list[0]
k = metadata[metadata['id'] == article_id][0:1]
key_list = list(k['keyword_list'])[0]
magazine_id = list(k['magazine_id'])[0]
key_list
magazine_id


0

In [ ]:
magazine_id = 0

In [203]:
list(magazine[magazine['id'] == 0]['magazine_tag_list'])

[[]]

In [207]:
len((list(magazine[magazine['id'] == 0]['magazine_tag_list']))[0])

0

In [214]:
read_list_len = []
for i in range(3000):
    read_list_len.append(len(user_read_articles[i]))
    

In [224]:
len(random.choices(read_list_len,k=200))

200

In [220]:
import random

In [225]:
len(user_read_articles) 
# unique한 article 수는 약 17만개를 읽음 (3천명이 읽은 글들 중)

user_matrix = np.zeros(shape=(3000,128),dtype = 'float32')

for i in tqdm(range(3000),desc='progress',mininterval=120):
    user_vector = np.zeros(shape=(128,),dtype='float32')
    user = user_id[i]
    read_list = user_read_articles[i]
    if(len(read_list) > 200):
        read_list = random.choices(read_list,k=200)
    cnt = 0
    for article_id in read_list:
        k = metadata[metadata['id'] == article_id][0:1]
        key_list = list(k['keyword_list'])[0]
        magazine_id = list(k['magazine_id'])[0]
        
        if len(key_list) != 0 :
            for word in key_list :
                user_vector += final_embeddings[word_to_id[word]]
                cnt +=1
        if magazine_id != 0:
            mag_list = list(magazine[magazine['id'] == magazine_id]['magazine_tag_list'])
            if len(mag_list) != 0:
                if len(mag_list[0]) != 0:
                    for word in mag_list[0]:
                        user_vector += final_embeddings[word_to_id[word]]
                        cnt+=1
    if(cnt != 0):
        user_vector /= cnt
    user_matrix[i] = user_vector
        
        
            





progress:   0%|                                                                               | 0/3000 [00:00<?, ?it/s]



progress:   0%|▎                                                                   | 15/3000 [02:04<6:53:54,  8.32s/it]



progress:   0%|▎                                                                   | 15/3000 [02:21<6:53:54,  8.32s/it]



progress:   1%|▌                                                                   | 24/3000 [04:06<8:10:15,  9.88s/it]



progress:   1%|▌                                                                   | 24/3000 [04:22<8:10:15,  9.88s/it]



progress:   1%|▊                                                                   | 38/3000 [06:25<8:08:21,  9.89s/it]



progress:   1%|▊                                                                   | 38/3000 [06:43<8:08:21,  9.89s/it]



progress:   2%|█                                                                   | 49/3000 [08:37<8:37:20, 10.52s/it]



progress:   

KeyboardInterrupt: 

In [187]:
len(user_vector)

128

In [188]:
cnt

1114

In [128]:
a = user_df['read_articles'][2]

In [130]:
import ast
a_li = ast.literal_eval(a)
a_li

['@yellowduck_64',
 '@yellowduck_63',
 '@minigorae_240',
 '@minigorae_179',
 '@minigorae_183',
 '@minigorae_256',
 '@minigorae_236',
 '@minigorae_249',
 '@minigorae_239',
 '@jiholim_82',
 '@minigorae_245',
 '@minigorae_235',
 '@minigorae_242',
 '@minigorae_243',
 '@minigorae_248',
 '@minigorae_244',
 '@minigorae_233',
 '@brunch_141',
 '@imaudio_39',
 '@egg0001_132',
 '@masism_334',
 '@minigorae_237',
 '@minigorae_254',
 '@minigorae_255',
 '@minigorae_246',
 '@sdain-sygang_29',
 '@minigorae_234',
 '@minigorae_258',
 '@minigorae_252',
 '@minigorae_257',
 '@minigorae_238',
 '@minigorae_241',
 '@minigorae_253',
 '@minigorae_247',
 '@minigorae_250']

In [79]:
print(len(total_top_50000))
print(len(recent_top_5000))
print(len(article_20190222_20190301))
print(len(article_20190301_20190314))

50000
5000
5189
23821


In [80]:
total_top_50000[:1]

,article_id,read_cnt,id,display_url,keyword_list,magazine_id,reg_ts,sub_title,title,author_id,reg_datetime,reg_dt,type,class
63061,@brunch_141,97206,141.0,https://brunch.co.kr/@brunch/141,"[브런치X빨강머리앤, 빨강머리앤, 출판, 작가]",0.0,1.539742e+12,글·그림 작가 파트너 프로젝트,브런치 작가가 함께 빨강머리 앤을 그리고 쓰다.,@brunch,2018-10-17 11:11:22,2018-10-17,개인,5%


643104

In [ ]:
# Step 6: embeddings 시각화

def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
    assert low_dim_embs.shape[0] >= len(labels), 'More labels than embeddings'

    plt.figure(figsize=(18, 18))        # in inches

    for (x, y), label in zip(low_dim_embs, labels):
        plt.scatter(x, y)
        plt.annotate(label,
                     xy=(x, y),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')

    plt.savefig(filename)

try:
    from sklearn.manifold import TSNE
    import matplotlib.pyplot as plt

    tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)

    plot_only = 500
    low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only])     # (500, 2)
    labels = ordered_words[:plot_only]                                  # 재구성한 코드

    plot_with_labels(low_dim_embs, labels)

except ImportError:
    print('Please install sklearn, matplotlib, and scipy to show embeddings.')

In [54]:
# 어떤 단어들 뽑혔는지 확인
li = []
for key in word_to_id:
    li.append(key)
li_pd = pd.DataFrame(li)
li_pd.to_csv(directory+'li_pd.csv',encoding='ms949')

## 참고자료들

- python list : https://www.w3schools.com/python/python_ref_list.asp
- 한국어 어휘 list : https://www.korean.go.kr/front/etcData/etcDataView.do?mn_id=46&etc_seq=71
- pandas to csv : https://mjdeeplearning.tistory.com/41
- dict to pandas : https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.from_dict.html
- dict to list : https://stackoverflow.com/questions/1679384/converting-dictionary-to-list
- for dict : https://wikidocs.net/16043
- list to dict : https://thispointer.com/python-pandas-how-to-convert-lists-to-a-dataframe/

# Skip Gram 구현시 참고자료
- 원본 코드 : https://github.com/tensorflow/tensorflow/blob/r1.2/tensorflow/examples/tutorials/word2vec/word2vec_basic.py // 원본 코드
- 설명 글 : https://github.com/YBIGTA/Deep_learning/blob/master/RNN/nlp/%EA%B5%AC%ED%98%84/skip-gram%20with%20tensorflow.md // 설명 불충분
    
- 완전 꼼꼼한 설명 글 : https://pythonkim.tistory.com/93 (매우 꼼꼼 +++)

# Tensorflow

- model save  : https://goodtogreate.tistory.com/entry/Saving-and-Restoring

# Numpy
- save, load : https://rfriend.tistory.com/358
        

In [17]:
batch = np.ndarray(shape=(8), dtype=np.int32)
labels = np.ndarray(shape=(8, 1), dtype=np.int32)

batch

array([-1932817184,         317,         116,          99,         104,
               317, -1288904656,         317])

In [18]:
labels

array([[0],
       [1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7]])